# 서울시 편의점 매출 결정요인 분석
## 05. 결과 해석 및 결론

---

### 이 노트북의 목표
1. 분석 결과 종합
2. 연구 질문에 대한 답변
3. 실무적 시사점 도출
4. 분석의 한계 인식

---

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

# 데이터 및 모델 재현
df = pd.read_csv('./분석데이터.csv', encoding='utf-8-sig')
df['발달상권'] = (df['주요_상권유형'] == '발달상권').astype(int)
df['전통시장'] = (df['주요_상권유형'] == '전통시장').astype(int)
df['관광특구'] = (df['주요_상권유형'] == '관광특구').astype(int)
df['미분류'] = (df['주요_상권유형'] == '미분류').astype(int)

y = df['당월_매출_금액']
X = df[['점포_수', '총_유동인구_수', '발달상권', '전통시장', '관광특구', '미분류']]
model = sm.OLS(y, sm.add_constant(X)).fit()

print(f"분석 데이터: {len(df):,}건")
print(f"R²: {model.rsquared:.4f}")

분석 데이터: 6,097건
R²: 0.6881


---
## 1. 연구 질문에 대한 답변

### 질문: 서울시 편의점 매출을 결정하는 핵심 요인은 무엇인가?

In [2]:
print("=" * 65)
print("연구 질문: 서울시 편의점 매출을 결정하는 핵심 요인은 무엇인가?")
print("=" * 65)

print("""
분석 결과, 편의점 매출에 통계적으로 유의한 영향을 미치는 요인은
다음 세 가지로 확인되었습니다:
""")

# 영향력 순위
factors = [
    ('점포수', model.params['점포_수']/1e8, '점포 1개 증가'),
    ('유동인구', model.params['총_유동인구_수']*1e6/1e8, '유동인구 100만명 증가'),
]

# 유의한 상권유형 추가
for v in ['발달상권', '관광특구', '전통시장']:
    if model.pvalues[v] < 0.05:
        factors.append((v, model.params[v]/1e8, f'{v} (vs 골목상권)'))

factors.sort(key=lambda x: abs(x[1]), reverse=True)

print("[영향력 순위] (매출 증가 효과, 억원)")
print("-" * 50)
for i, (name, effect, desc) in enumerate(factors, 1):
    print(f"  {i}. {desc}: {effect:+.2f}억원")

연구 질문: 서울시 편의점 매출을 결정하는 핵심 요인은 무엇인가?

분석 결과, 편의점 매출에 통계적으로 유의한 영향을 미치는 요인은
다음 세 가지로 확인되었습니다:

[영향력 순위] (매출 증가 효과, 억원)
--------------------------------------------------
  1. 관광특구 (vs 골목상권): +27.19억원
  2. 발달상권 (vs 골목상권): +14.72억원
  3. 전통시장 (vs 골목상권): +11.94억원
  4. 점포 1개 증가: +3.99억원
  5. 유동인구 100만명 증가: +1.86억원


---
## 2. 가설별 결론

In [3]:
print("=" * 65)
print("가설별 최종 결론")
print("=" * 65)

# H1
print("""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
H1: 유동인구가 많을수록 매출이 높다
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
""")
c1, p1 = model.params['총_유동인구_수'], model.pvalues['총_유동인구_수']
print(f"  결과: 채택 (p < 0.001)")
print(f"  효과: 유동인구 100만명 증가 시 매출 {c1*1e6/1e8:.2f}억원 증가")
print(f"")
print(f"  해석: 유동인구가 많은 지역은 잠재 고객이 많아 매출이 높다.")
print(f"        역세권, 대학가, 오피스 밀집 지역이 유리하다.")

# H2  
print("""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
H2: 점포수(밀집도)가 높을수록 매출이 높다
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
""")
c2, p2 = model.params['점포_수'], model.pvalues['점포_수']
print(f"  결과: 채택 (p < 0.001)")
print(f"  효과: 점포 1개 증가 시 매출 {c2/1e8:.2f}억원 증가")
print(f"")
print(f"  해석: 점포가 많이 몰린 지역은 총 매출이 높다.")
print(f"        단, 이는 '수요가 있어서 점포가 생긴 것'일 수 있다 (역인과).")
print(f"        점포가 많다고 개별 점포 매출이 높은 건 아님에 주의.")

# H3
print("""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
H3: 상권유형에 따라 매출 차이가 있다
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
""")
print(f"  결과: 채택 (유의한 차이 존재)")
print(f"  기준: 골목상권")
print(f"")
print(f"  상권유형별 매출 차이:")
for v in ['발달상권', '관광특구', '전통시장', '미분류']:
    c, p = model.params[v], model.pvalues[v]
    sig = '*' if p < 0.05 else ''
    d = '높음' if c > 0 else '낮음'
    print(f"    {v}: 골목상권 대비 {abs(c)/1e8:.2f}억원 {d} {sig}")
print(f"")
print(f"  해석: 상권유형에 따라 매출 격차가 뚜렷하다.")
print(f"        발달상권이 가장 유리하며, 미분류(주거지역)가 가장 불리하다.")

가설별 최종 결론

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
H1: 유동인구가 많을수록 매출이 높다
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

  결과: 채택 (p < 0.001)
  효과: 유동인구 100만명 증가 시 매출 1.86억원 증가

  해석: 유동인구가 많은 지역은 잠재 고객이 많아 매출이 높다.
        역세권, 대학가, 오피스 밀집 지역이 유리하다.

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
H2: 점포수(밀집도)가 높을수록 매출이 높다
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

  결과: 채택 (p < 0.001)
  효과: 점포 1개 증가 시 매출 3.99억원 증가

  해석: 점포가 많이 몰린 지역은 총 매출이 높다.
        단, 이는 '수요가 있어서 점포가 생긴 것'일 수 있다 (역인과).
        점포가 많다고 개별 점포 매출이 높은 건 아님에 주의.

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
H3: 상권유형에 따라 매출 차이가 있다
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

  결과: 채택 (유의한 차이 존재)
  기준: 골목상권

  상권유형별 매출 차이:
    발달상권: 골목상권 대비 14.72억원 높음 *
    관광특구: 골목상권 대비 27.19억원 높음 *
    전통시장: 골목상권 대비 11.94억원 높음 *
    미분류: 골목상권 대비 4.39억원 높음 *

  해석: 상권유형에 따라 매출 격차가 뚜렷하다.
        발달상권이 가장 유리하며,

---
## 3. 실무적 시사점

In [4]:
print("=" * 65)
print("실무적 시사점")
print("=" * 65)

print("""
┌─────────────────────────────────────────────────────────────────┐
│  1. 신규 출점 시 입지 선정 기준                                  │
└─────────────────────────────────────────────────────────────────┘

  [우선순위 1] 상권유형 확인
    - 발달상권 > 관광특구 > 전통시장 > 골목상권 순으로 유리
    - 같은 조건이면 발달상권 입지 선택

  [우선순위 2] 유동인구 확인  
    - 유동인구 데이터를 활용해 잠재 수요 추정
    - 역세권, 대학가, 오피스 밀집 지역 우선 검토

  [주의사항] 점포 밀집도
    - 점포가 많은 지역 = 수요가 있는 지역
    - 그러나 경쟁 강도도 높으므로 신중한 판단 필요

""")

print("""
┌─────────────────────────────────────────────────────────────────┐
│  2. 기존 점포 운영 전략                                         │
└─────────────────────────────────────────────────────────────────┘

  [발달상권/관광특구]
    - 높은 유동인구 활용: 충동구매 유도 상품 전진 배치
    - 관광특구: 외국인 대상 상품, 기념품 구비
    - 프리미엄 상품 비중 확대

  [골목상권/전통시장]
    - 생활 밀착형 상품 강화 (생필품, 식료품)
    - 배달 서비스 도입 검토
    - 단골 고객 관리 (멤버십, 적립)

  [미분류(주거지역)]
    - 가장 매출이 낮은 유형
    - 야간/새벽 시간대 틈새 수요 공략
    - 택배 수령, 공과금 납부 등 부가 서비스 강화
""")

실무적 시사점

┌─────────────────────────────────────────────────────────────────┐
│  1. 신규 출점 시 입지 선정 기준                                  │
└─────────────────────────────────────────────────────────────────┘

  [우선순위 1] 상권유형 확인
    - 발달상권 > 관광특구 > 전통시장 > 골목상권 순으로 유리
    - 같은 조건이면 발달상권 입지 선택

  [우선순위 2] 유동인구 확인  
    - 유동인구 데이터를 활용해 잠재 수요 추정
    - 역세권, 대학가, 오피스 밀집 지역 우선 검토

  [주의사항] 점포 밀집도
    - 점포가 많은 지역 = 수요가 있는 지역
    - 그러나 경쟁 강도도 높으므로 신중한 판단 필요



┌─────────────────────────────────────────────────────────────────┐
│  2. 기존 점포 운영 전략                                         │
└─────────────────────────────────────────────────────────────────┘

  [발달상권/관광특구]
    - 높은 유동인구 활용: 충동구매 유도 상품 전진 배치
    - 관광특구: 외국인 대상 상품, 기념품 구비
    - 프리미엄 상품 비중 확대

  [골목상권/전통시장]
    - 생활 밀착형 상품 강화 (생필품, 식료품)
    - 배달 서비스 도입 검토
    - 단골 고객 관리 (멤버십, 적립)

  [미분류(주거지역)]
    - 가장 매출이 낮은 유형
    - 야간/새벽 시간대 틈새 수요 공략
    - 택배 수령, 공과금 납부 등 부가 서비스 강화



---
## 4. 분석의 한계

In [5]:
print("=" * 65)
print("분석의 한계")
print("=" * 65)

print("""
[1] 변수의 한계
    - 포함된 변수: 유동인구, 점포수, 상권유형
    - 미포함 변수: 임대료, 경쟁 강도, 점포 면적, 브랜드, 운영시간 등
    - 미포함 변수가 매출에 영향을 줄 수 있음

[2] 분석 단위의 한계
    - 분석 단위: 행정동 × 분기
    - 개별 점포 수준의 분석이 아님
    - 행정동 내 점포 간 편차는 반영되지 않음

[3] 인과관계 해석 주의
    - 회귀분석은 상관관계를 보여줄 뿐, 인과관계를 증명하지 않음
    - 예: "점포가 많아서 매출이 높다" vs "매출이 높아서 점포가 많다"
    - 역인과 가능성 존재

[4] 시계열 특성 미반영
    - 패널 데이터이나 단순 OLS 사용
    - 고정효과 모형 미적용
    - 시간에 따른 변화 패턴이 충분히 반영되지 않음

[5] 외부 요인 미반영
    - 코로나19 영향 (2022년 초)
    - 물가 상승, 소비 트렌드 변화
    - 계절성 (명시적 모델링 미포함)
""")

분석의 한계

[1] 변수의 한계
    - 포함된 변수: 유동인구, 점포수, 상권유형
    - 미포함 변수: 임대료, 경쟁 강도, 점포 면적, 브랜드, 운영시간 등
    - 미포함 변수가 매출에 영향을 줄 수 있음

[2] 분석 단위의 한계
    - 분석 단위: 행정동 × 분기
    - 개별 점포 수준의 분석이 아님
    - 행정동 내 점포 간 편차는 반영되지 않음

[3] 인과관계 해석 주의
    - 회귀분석은 상관관계를 보여줄 뿐, 인과관계를 증명하지 않음
    - 예: "점포가 많아서 매출이 높다" vs "매출이 높아서 점포가 많다"
    - 역인과 가능성 존재

[4] 시계열 특성 미반영
    - 패널 데이터이나 단순 OLS 사용
    - 고정효과 모형 미적용
    - 시간에 따른 변화 패턴이 충분히 반영되지 않음

[5] 외부 요인 미반영
    - 코로나19 영향 (2022년 초)
    - 물가 상승, 소비 트렌드 변화
    - 계절성 (명시적 모델링 미포함)



---
## 5. 최종 결론

In [6]:
print("="*70)
print("최종 결론")
print("="*70)

n_dong = df['행정동_코드'].nunique()
n_qtr = df['기준_년분기_코드'].nunique()
q_min = df['기준_년분기_코드'].min()
q_max = df['기준_년분기_코드'].max()

print(f"""
본 연구는 서울시 편의점 매출에 영향을 미치는 요인을 분석하였다.

┌─────────────────────────────────────────────────────────────────┐
│  분석 개요                                                      │
├─────────────────────────────────────────────────────────────────┤
│  기간: {q_min//10}년 {q_min%10}분기 ~ {q_max//10}년 {q_max%10}분기 ({n_qtr}개 분기)             │
│  범위: 서울시 {n_dong}개 행정동                                       │
│  관측치: {len(df):,}건                                               │
│  방법: 다중회귀분석 (OLS)                                        │
└─────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────┐
│  핵심 발견                                                      │
├─────────────────────────────────────────────────────────────────┤
│  1. 유동인구, 점포수, 상권유형 모두 매출에 유의한 영향           │
│  2. 점포 1개 증가 시 약 {model.params['점포_수']/1e8:.1f}억원 매출 증가               │
│  3. 유동인구 100만명 증가 시 약 {model.params['총_유동인구_수']*1e6/1e8:.1f}억원 매출 증가       │
│  4. 발달상권이 골목상권 대비 약 {model.params['발달상권']/1e8:.1f}억원 높은 매출         │
│  5. 모델 설명력 R² = {model.rsquared:.2f} (매출 변동의 {model.rsquared*100:.0f}% 설명)       │
└─────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────┐
│  결론                                                           │
├─────────────────────────────────────────────────────────────────┤
│  편의점 매출은 입지 특성에 크게 좌우된다.                        │
│  특히 상권유형과 유동인구가 핵심 결정요인이다.                   │
│  신규 출점 시 발달상권, 고유동인구 지역을 우선 검토해야 한다.    │
└─────────────────────────────────────────────────────────────────┘
""")

최종 결론

본 연구는 서울시 편의점 매출에 영향을 미치는 요인을 분석하였다.

┌─────────────────────────────────────────────────────────────────┐
│  분석 개요                                                      │
├─────────────────────────────────────────────────────────────────┤
│  기간: 2022년 1분기 ~ 2025년 3분기 (15개 분기)             │
│  범위: 서울시 414개 행정동                                       │
│  관측치: 6,097건                                               │
│  방법: 다중회귀분석 (OLS)                                        │
└─────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────┐
│  핵심 발견                                                      │
├─────────────────────────────────────────────────────────────────┤
│  1. 유동인구, 점포수, 상권유형 모두 매출에 유의한 영향           │
│  2. 점포 1개 증가 시 약 4.0억원 매출 증가               │
│  3. 유동인구 100만명 증가 시 약 1.9억원 매출 증가       │
│  4. 발달상권이 골목상권 대비 약 14.7억원 높은 매출         │
│  5. 모델 설명력 R² = 0.69 (매출 변동의 69% 설명)       │
└──────────────────

---
## 분석 완료

In [7]:
print("""
╔═════════════════════════════════════════════════════════════════╗
║                      분석 노트북 구성                           ║
╠═════════════════════════════════════════════════════════════════╣
║  01_전처리.ipynb   │ 데이터 통합 및 변수 생성                   ║
║  02_EDA.ipynb      │ 가설별 탐색적 분석                         ║
║  03_시각화.ipynb   │ 가설별 시각적 검증                         ║
║  04_회귀분석.ipynb │ 가설 통계적 검증 (OLS)                     ║
║  05_해석.ipynb     │ 결과 종합 및 시사점 (현재)                 ║
╚═════════════════════════════════════════════════════════════════╝

분석 완료. 감사합니다.
""")


╔═════════════════════════════════════════════════════════════════╗
║                      분석 노트북 구성                           ║
╠═════════════════════════════════════════════════════════════════╣
║  01_전처리.ipynb   │ 데이터 통합 및 변수 생성                   ║
║  02_EDA.ipynb      │ 가설별 탐색적 분석                         ║
║  03_시각화.ipynb   │ 가설별 시각적 검증                         ║
║  04_회귀분석.ipynb │ 가설 통계적 검증 (OLS)                     ║
║  05_해석.ipynb     │ 결과 종합 및 시사점 (현재)                 ║
╚═════════════════════════════════════════════════════════════════╝

분석 완료. 감사합니다.

